In [1]:
import cv2
import os

from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from vllm.assets.image import ImageAsset 
from vllm.assets.video import VideoAsset
from vllm.utils import FlexibleArgumentParser
# Specify the maximum number of frames per video to be 4. This can be changed.
# llm = LLM("Qwen/Qwen2.5-VL-3B-Instruct", limit_mm_per_prompt={"image": 4}, max_model_len=10000)
llm = LLM(
        model="Qwen/Qwen2.5-VL-7B-Instruct",
        max_model_len=8000,
        max_num_seqs=10)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-02-23 22:41:13.781058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740350473.789359  153707 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740350473.793206  153707 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 22:41:15,536	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook ou

INFO 02-23 22:41:15 __init__.py:207] Automatically detected platform cuda.
INFO 02-23 22:41:22 config.py:560] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
INFO 02-23 22:41:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.4.dev53+geb24dc4a.d20250223) with config: model='Qwen/Qwen2.5-VL-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 02-23 22:42:15 model_runner.py:1117] Loading model weights took 15.6270 GB and 51.107047 seconds


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


WARNING 02-23 22:42:15 model_runner.py:1291] Computed max_num_seqs (min(10, 8000 // 32768)) to be less than 1. Setting it to the minimum value of 1.


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


WARNING 02-23 22:42:19 profiling.py:196] The context length (8000) of the model is too short to hold the multi-modal embeddings in the worst case (32768 tokens in total, out of which {'image': 16384, 'video': 16384} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
INFO 02-23 22:42:21 worker.py:267] Memory profiling takes 5.45 seconds
INFO 02-23 22:42:21 worker.py:267] the current vLLM instance can use total_gpu_memory (94.50GiB) x gpu_memory_utilization (0.90) = 85.05GiB
INFO 02-23 22:42:21 worker.py:267] model weights take 15.63GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 4.38GiB; the rest of the memory reserved for KV Cache is 64.90GiB.
INFO 02-23 22:42:21 executor_base.py:111] # cuda blocks: 75948, # CPU blocks: 4681
INFO 02-23 22:42:21 executor_base.py:116] M

Capturing CUDA graph shapes: 100%|██████████| 5/5 [00:02<00:00,  1.94it/s]

INFO 02-23 22:42:32 model_runner.py:1565] Graph capturing finished in 3 secs, took 0.18 GiB
INFO 02-23 22:42:32 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 17.13 seconds


In [2]:
sampling_params = SamplingParams(temperature=0.7,
                                     max_tokens=256)

In [3]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("HuggingFaceFV/finevideo", split="train")

# Access the first video
video = ds[0]["mp4"]

#full dataset (600GB of data)
#dataset = load_dataset("HuggingFaceFV/finevideo/", split="train", num_proc=24)


Resolving data files:   0%|          | 0/1357 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1357 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/914 [00:00<?, ?it/s]

In [4]:
# 1. Specify a dataset of video (100 videos)
# 2. Specify a prompt that asks the model to generate a description of the video * 100
# 3. You should be able to feed all of it together at once to the model to generate (DO NOT FEED ONE BY ONE)

In [5]:
from decord import VideoReader, cpu
from PIL import Image
import matplotlib.pyplot as plt 
from tqdm import tqdm
import numpy as np 

In [6]:

# Load the videos 
dataset_path = "/home/ubuntu/temp/10k_vid_dataset/mp4s"
os.makedirs(dataset_path, exist_ok=True)

num_videos = 2500
videos = []
num_chunks = 2
time_per_chunk = 10
for i in tqdm(range(num_videos), total=num_videos):
    #save video 
    video_path = os.path.join(dataset_path, f"video_{i}.mp4")
    with open(video_path, "wb") as f:
        f.write(ds[i]["mp4"])
    # read video    
    vr = VideoReader(video_path, ctx=cpu(0))
    vid_dict = {"vid_path": video_path}
    success = True
    for chunk_idx in range(num_chunks):
        try:
            frame_idxs = np.arange(chunk_idx * time_per_chunk * 30, (chunk_idx + 1) * time_per_chunk * 30, 30)
            nd_frames = vr.get_batch(frame_idxs).asnumpy()
            vid_dict[f"vT{chunk_idx}"] = nd_frames
        except:
            print(f"Error processing")
            success = False
    if(success):
        videos.append(vid_dict)

  1%|▏         | 34/2500 [00:03<03:29, 11.75it/s]

Error processing


  3%|▎         | 69/2500 [00:07<04:44,  8.55it/s][h264 @ 0xb13bae6806c0] mmco: unref short failure
[h264 @ 0xb13bae6806c0] mmco: unref short failure
  7%|▋         | 173/2500 [00:20<04:29,  8.63it/s][h264 @ 0xb13bafb2f290] mmco: unref short failure
[h264 @ 0xb13bafb2f290] mmco: unref short failure
  8%|▊         | 199/2500 [00:23<04:45,  8.05it/s][h264 @ 0xb13bc8922e20] mmco: unref short failure
[h264 @ 0xb13bc8922e20] mmco: unref short failure
 10%|▉         | 248/2500 [00:28<03:10, 11.82it/s][h264 @ 0xb13bb0ff6680] mmco: unref short failure
[h264 @ 0xb13bb0ff6680] mmco: unref short failure
 10%|█         | 262/2500 [00:29<04:14,  8.78it/s][h264 @ 0xb13bae6f5230] mmco: unref short failure
[h264 @ 0xb13bae6f5230] mmco: unref short failure
[h264 @ 0xb13bae6f5230] mmco: unref short failure
[h264 @ 0xb13bae6f5230] mmco: unref short failure
 12%|█▏        | 292/2500 [00:33<04:15,  8.64it/s][h264 @ 0xb13bae6857f0] mmco: unref short failure
[h264 @ 0xb13bae6857f0] mmco: unref short failure
 

Error processing


 50%|█████     | 1250/2500 [02:19<01:52, 11.13it/s]

...

 55%|█████▍    | 1368/2500 [02:34<02:49,  6.69it/s][h264 @ 0xb13c06353050] mmco: unref short failure
[h264 @ 0xb13c06353050] mmco: unref short failure
 64%|██████▍   | 1600/2500 [03:02<01:29, 10.10it/s][h264 @ 0xb13c062eb210] mmco: unref short failure
[h264 @ 0xb13c062eb210] mmco: unref short failure
 65%|██████▌   | 1631/2500 [03:06<02:05,  6.93it/s][h264 @ 0xb13bb0fbc050] mmco: unref short failure
[h264 @ 0xb13bb0fbc050] mmco: unref short failure
 68%|██████▊   | 1701/2500 [03:15<01:29,  8.92it/s][h264 @ 0xb13d8a224ee0] mmco: unref short failure
[h264 @ 0xb13d8a224ee0] mmco: unref short failure
 77%|███████▋  | 1937/2500 [03:37<00:56,  9.90it/s][h264 @ 0xb13bae652730] mmco: unref short failure
[h264 @ 0xb13bae652730] mmco: unref short failure
 83%|████████▎ | 2064/2500 [03:52<01:03,  6.88it/s][h264 @ 0xb13bae5c3d50] mmco: unref short failure
[h264 @ 0xb13bae5c3d50] mmco: unref short failure
 84%|████████▍ | 2095/2500 [03:56<00:45,  8.85it/s][h264 @ 0xb13bae885150] mmco: unref short f

Error processing


 93%|█████████▎| 2320/2500 [04:23<00:15, 11.69it/s]

Error processing


100%|██████████| 2500/2500 [04:45<00:00,  8.75it/s]


In [7]:
# DATA STRUCTURE
# videos = [{"vid_path": "path...",
#            "vid": [frame1, frame2, frame3, ...],
#            "T1": {"description": {"Q": "text...", "A": "text..."},
#                   "temporal": {"time": int, "content": "text..."}},
#                   "question": {"Q": "text...", "A": "text..."},
#           }]

In [7]:
from vllm.assets.video import VideoAsset

def generate_video_description(video_frame_list, prompt, num_chunks):
    messages = []
    for idx, dict_obj in enumerate(video_frame_list):
        for chunk_idx in range(num_chunks):
            if(type(prompt) == list):
                prompt_curr = prompt[idx*num_chunks + chunk_idx]
            else:
                prompt_curr = prompt
            message = {"prompt": prompt_curr, "multi_modal_data": {"video": dict_obj[f"vT{chunk_idx}"]}}
            messages.append(message)
    # Perform inference and log output.
    print("Starting inference on ", len(messages), " messages")
    outputs = llm.generate(messages, sampling_params=sampling_params)
    generated_texts = []
    for idx in range(len(outputs)):
        generated_text = outputs[idx].outputs[0].text
        generated_texts.append(generated_text)
    print("Generated", len(generated_texts), "texts")
    return generated_texts
    

# Description Generation

In [8]:
prompt_describe = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              f"Describe what is shown in the video. <|im_end|>\n"
              "<|im_start|>assistant\n")

generated_texts = generate_video_description(videos, prompt_describe, num_chunks)

for idx, text in enumerate(generated_texts):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    start_time = chunk_idx * time_per_chunk
    end_time = (chunk_idx + 1) * time_per_chunk
    print(f"Video {list_idx} chunk {chunk_idx} time {start_time} to {end_time} seconds")
    videos[list_idx][f"T{chunk_idx}_description"] = {"Q": f"What occurs between {start_time} and {end_time} seconds?", "A": text}

Starting inference on  4990  messages


Processed prompts: 100%|██████████| 4990/4990 [18:52<00:00,  4.41it/s, est. speed input: 6418.99 toks/s, output: 681.97 toks/s]

Generated 4990 texts
Video 0 chunk 0 time 0 to 10 seconds
Video 0 chunk 1 time 10 to 20 seconds
Video 1 chunk 0 time 0 to 10 seconds
Video 1 chunk 1 time 10 to 20 seconds
Video 2 chunk 0 time 0 to 10 seconds
Video 2 chunk 1 time 10 to 20 seconds
Video 3 chunk 0 time 0 to 10 seconds
Video 3 chunk 1 time 10 to 20 seconds
Video 4 chunk 0 time 0 to 10 seconds
Video 4 chunk 1 time 10 to 20 seconds
Video 5 chunk 0 time 0 to 10 seconds
Video 5 chunk 1 time 10 to 20 seconds
Video 6 chunk 0 time 0 to 10 seconds
Video 6 chunk 1 time 10 to 20 seconds
Video 7 chunk 0 time 0 to 10 seconds
Video 7 chunk 1 time 10 to 20 seconds
Video 8 chunk 0 time 0 to 10 seconds
Video 8 chunk 1 time 10 to 20 seconds
Video 9 chunk 0 time 0 to 10 seconds
Video 9 chunk 1 time 10 to 20 seconds
Video 10 chunk 0 time 0 to 10 seconds
Video 10 chunk 1 time 10 to 20 seconds
Video 11 chunk 0 time 0 to 10 seconds
Video 11 chunk 1 time 10 to 20 seconds
Video 12 chunk 0 time 0 to 10 seconds
Video 12 chunk 1 time 10 to 20 second

In [9]:
print(videos[-1]["vid_path"])
for key in videos[-1].keys():
    if("description" in key):
        print(f"{key}: {videos[-1][key]}")

/home/ubuntu/temp/10k_vid_dataset/mp4s/video_2499.mp4
T0_description: {'Q': 'What occurs between 0 and 10 seconds?', 'A': 'The video shows a group of young women dressed in black jackets and black caps, all wearing masks. They are standing on a track field in front of a set of bleachers. The women appear to be part of a team, as their outfits are uniform. They are interacting with each other, some smiling and some looking more serious, suggesting a casual or pre-game setting. The environment appears to be outdoors, with clear daylight. There is no prominent text or logos visible on the screen except for the watermark "HCAMNews" in the corner.'}
T1_description: {'Q': 'What occurs between 10 and 20 seconds?', 'A': 'The video shows a group of people participating in an event on a track, likely a cross-country or running event. The individuals are dressed in athletic clothing, including sneakers, leggings, and jackets, with some wearing long-sleeved shirts and others with visible bright le

# Time Step Generation

In [10]:
prompt_timestep = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              "What occurs at time {time} seconds?<|im_end|>\n"
              "<|im_start|>assistant\n")

sample_time_steps = np.random.randint(0, time_per_chunk, num_videos*num_chunks)

prompt_list_timesteps = []
for time_step in sample_time_steps:
    prompt = prompt_timestep.replace("{time}", str(time_step))
    prompt_list_timesteps.append(prompt)

generated_texts = generate_video_description(videos, prompt_list_timesteps, num_chunks)

for idx, text in enumerate(generated_texts):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    vid_time = sample_time_steps[idx]
    calc_time = chunk_idx*time_per_chunk+sample_time_steps[idx] # have to recalculate due to chunking
    videos[list_idx][f"T{chunk_idx}_timestep"] = {"Q:": f"What occurs at time {calc_time} seconds?", "A": text.replace(str(vid_time), str(calc_time))}

Starting inference on  4990  messages


Processed prompts: 100%|██████████| 4990/4990 [12:37<00:00,  6.59it/s, est. speed input: 9594.47 toks/s, output: 471.56 toks/s]

Generated 4990 texts


In [11]:
print(videos[0].keys())
for key in videos[0].keys():
    if("timestep" in key):
        print(f"{key}: {videos[0][key]}")

dict_keys(['vid_path', 'vT0', 'vT1', 'T0_description', 'T1_description', 'T0_timestep', 'T1_timestep'])
T0_timestep: {'Q:': 'What occurs at time 5 seconds?', 'A': 'At 5 seconds, the video transitions from the sign for the "Heavy Equipment Operator Training Centre" to a view of a construction site.'}
T1_timestep: {'Q:': 'What occurs at time 10 seconds?', 'A': 'At time 10 seconds, the video begins with a scene showing a construction site. The background includes trees and what appears to be a forested area, with some construction equipment and vehicles visible in the distance. The scene then transitions to a woman in an orange safety vest standing in the same location, likely at a construction site. The text overlay identifies her as "Sara Paynton," associated with "SICA/Velocity Client."'}


# QA

In [12]:
prompt_gen_question =  ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              f"What is a vision understanding question you can ask about the visual content of the video. Only output the question<|im_end|>\n"
              "<|im_start|>assistant\n")

generated_texts = generate_video_description(videos, prompt_gen_question, num_chunks)

questions = generated_texts

Starting inference on  4990  messages


Processed prompts: 100%|██████████| 4990/4990 [08:38<00:00,  9.62it/s, est. speed input: 14127.50 toks/s, output: 167.03 toks/s]


Generated 4990 texts


In [13]:
prompt_answer_question =  ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              "{question}<|im_end|>\n"
              "<|im_start|>assistant\n")


prompt_qas = []
for q in questions:
    qa = prompt_answer_question.replace("{question}", q)
    prompt_qas.append(qa)


generated_texts = generate_video_description(videos, prompt_qas, num_chunks)
answers = generated_texts

for idx in range(len(answers)):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    q = questions[idx]
    a = answers[idx]
    videos[list_idx][f"T{chunk_idx}_qa"] = {"Q": q, "A": a}

Starting inference on  4990  messages


Processed prompts: 100%|██████████| 4990/4990 [14:30<00:00,  5.73it/s, est. speed input: 8392.78 toks/s, output: 544.65 toks/s]

Generated 4990 texts


In [14]:
vid_set = videos[-3]
print(vid_set.keys())
for key in vid_set.keys():
    if("qa" in key):
        print("Q", vid_set[key]["Q"])
        print("A", vid_set[key]["A"])
        print("______________")

dict_keys(['vid_path', 'vT0', 'vT1', 'T0_description', 'T1_description', 'T0_timestep', 'T1_timestep', 'T0_qa', 'T1_qa'])
Q What is the theme or message conveyed by the video that transitions from a library setting to a forest scene?
A The transition from a library setting to a forest scene in the video could symbolize a shift from intellectual pursuits to nature and the appreciation of the natural world. Libraries often represent knowledge and learning, while the forest scene, with the couple holding hands, could suggest themes of companionship, connection, and the simple joys found in nature. This could imply a balance between the pursuit of knowledge and the importance of personal relationships and moments of reflection in the natural environment.
______________
Q What is the purpose of the man and woman walking along the path in the forest?
A The man and woman appear to be walking along a path in the forest as part of a conversation or discussion, as they are holding hands and look

In [15]:
vid_set = videos[-8]
print(vid_set.keys())
print(vid_set["vid_path"])
for key in vid_set.keys():
    if("vT" not in key):
        print(key, vid_set[key])

dict_keys(['vid_path', 'vT0', 'vT1', 'T0_description', 'T1_description', 'T0_timestep', 'T1_timestep', 'T0_qa', 'T1_qa'])
/home/ubuntu/temp/10k_vid_dataset/mp4s/video_2492.mp4
vid_path /home/ubuntu/temp/10k_vid_dataset/mp4s/video_2492.mp4
T0_description {'Q': 'What occurs between 0 and 10 seconds?', 'A': 'The video begins with a black screen. The text "MONOLITH PICTURES" appears in the center of the screen in white capital letters. The letters "I" in "MONOLITH" are stylized with a vertical line, giving the O a unique shape. The text remains static for a moment before the screen fades to black again.'}
T1_description {'Q': 'What occurs between 10 and 20 seconds?', 'A': 'The video begins with a wide, desolate landscape, showing a flat, expansive area with low-lying terrain, possibly a desert or a barren wasteland. The sky is overcast, and the lighting is dim, creating a somber and bleak atmosphere. In the foreground, there are scattered rocks and small patches of low vegetation. \n\nAs t

In [16]:
#save to huggingface dictionary
from datasets import Dataset
root_path = "/home/ubuntu/temp/10k_vid_dataset/"
#recreate data list without video frames
data_list = []
for vid in videos:
    vid_dict = {}
    for key in vid.keys():
        if("vT" not in key):
            vid_dict[key] = vid[key]
    data_list.append(vid_dict)
#save to huggingface dataset
dataset = Dataset.from_list(data_list)
dataset.save_to_disk(os.path.join(root_path, "finevideo_dataset"))

Saving the dataset (0/1 shards):   0%|          | 0/2495 [00:00<?, ? examples/s]

In [17]:
test_load = Dataset.load_from_disk(os.path.join(root_path, "finevideo_dataset"))
print(test_load[0].keys())

dict_keys(['vid_path', 'T0_description', 'T1_description', 'T0_timestep', 'T1_timestep', 'T0_qa', 'T1_qa'])


In [18]:
len(test_load)

2495

In [20]:
test_load[-1]

{'vid_path': '/home/ubuntu/temp/10k_vid_dataset/mp4s/video_2499.mp4',
 'T0_description': {'A': 'The video shows a group of young women dressed in black jackets and black caps, all wearing masks. They are standing on a track field in front of a set of bleachers. The women appear to be part of a team, as their outfits are uniform. They are interacting with each other, some smiling and some looking more serious, suggesting a casual or pre-game setting. The environment appears to be outdoors, with clear daylight. There is no prominent text or logos visible on the screen except for the watermark "HCAMNews" in the corner.',
  'Q': 'What occurs between 0 and 10 seconds?'},
 'T1_description': {'A': 'The video shows a group of people participating in an event on a track, likely a cross-country or running event. The individuals are dressed in athletic clothing, including sneakers, leggings, and jackets, with some wearing long-sleeved shirts and others with visible bright leggings and head coveri